In [1]:
#Import all the dependencies
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from gensim.models.doc2vec import Doc2Vec

In [2]:
#Import training data from csv
df= pd.read_csv("final_dataset.csv")
data = df["Responses"].tolist()
#print(data)
tagged_data = [TaggedDocument(words=word_tokenize(str(_d).lower()), tags=[str(i)]) for i, _d in enumerate(data)]

In [3]:
max_epochs = [100]
vec_size = 10000
alpha = 0.025

for k in max_epochs:
    print("max_epochs:" + str(k))
          
    model = Doc2Vec(vector_size=vec_size,
                    alpha=alpha, 
                    min_alpha=0.00025,
                    min_count=1,
                    dm =1)

    model.build_vocab(tagged_data)

    for epoch in range(k):
        #print('iteration {0}'.format(epoch))
        model.train(tagged_data,
                    total_examples=model.corpus_count,
                    epochs=model.epochs)
        # decrease the learning rate
        model.alpha -= 0.0002
        # fix the learning rate, no decay
        model.min_alpha = model.alpha

    model.save("d2v.model")

    model= Doc2Vec.load("d2v.model")
    #to find the vector of a document which is not in training data
    f = open("reference_answer.txt", "r+")
    refAnswer = f.readline()
    f.close()

    ref_answer_data = word_tokenize(refAnswer.lower())
    ref_answer_vector = model.infer_vector(ref_answer_data)
    #print("V1_infer", ref_answer_vector)

    cos_sim = []
    for i in df["Responses"]:
        response_data = word_tokenize(str(i).lower())
        response_vector = model.infer_vector(response_data)
        dot = np.dot(response_vector, ref_answer_vector)
        norma = np.linalg.norm(response_vector)
        normb = np.linalg.norm(ref_answer_vector)
        cos = dot / (norma * normb)
        cos_sim.append(cos)
    df["cos_sim"] = cos_sim

    X = df['cos_sim'].values
    y = df['Final Score'].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3, random_state=0)

    # Fitting Simple Linear Regression to the Training set
    regressor = LinearRegression()
    regressor.fit(X_train.reshape(-1, 1), y_train.reshape(-1, 1))
    
    y_pred = regressor.predict(X_test.reshape(-1, 1))
    
    df1 = pd.DataFrame(y_test.reshape(-1, 1), columns = ['act score'])
    df1["prd_score"] = y_pred
    print(df1.head(100))
    
    """# Visualizing the Training set results
    viz_train = plt
    viz_train.scatter(X_train.reshape(-1, 1), y_train.reshape(-1, 1), color='red')
    viz_train.plot(X_train.reshape(-1, 1), regressor.predict(X_train.reshape(-1, 1)), color='blue')
    viz_train.title('Original score VS Cosin similarity (Training set)')
    viz_train.xlabel('Cosin similarity')
    viz_train.ylabel('Original score')
    viz_train.show()

    # Visualizing the Test set results
    viz_test = plt
    viz_test.scatter(X_test.reshape(-1, 1), y_test.reshape(-1, 1), color='red')
    viz_test.plot(X_train.reshape(-1, 1), regressor.predict(X_train.reshape(-1, 1)), color='blue')
    viz_test.title('Original score VS Cosin similarity (Test set)')
    viz_test.xlabel('Cosin similarity')
    viz_test.ylabel('Original score')
    viz_test.show()"""

max_epochs:100
    act score  prd_score
0          20  18.062727
1          15  18.094068
2           0  17.819857
3          25  17.215531
4          30  17.808391
5          15  17.694807
6          20  18.270608
7          25  17.774553
8          30  18.084802
9          20  17.045041
10         25  17.991106
11         15  17.960514
12         20  17.431810
13         15  18.037452
14          5  17.586563
15         20  17.405878
16         25  17.952830
17          0  18.049886
18         20  18.163231
19         30  17.420772
20         15  16.849757
21         20  18.045673
22         10  16.623462
23         15  17.829019
24         10  17.959496
25         15  16.791408
26         20  15.808602
27          5  17.813835
28         20  18.047356
29         10  18.198469
..        ...        ...
70         25  17.279722
71         20  18.070663
72          5  18.051421
73          5  17.861719
74         30  18.010212
75         10  17.764363
76         15  17.492229
77        